In [0]:
!pip install shap

     |████████████████████████████████| 276kB 3.5MB/s 
  Created wheel for shap: filename=shap-0.35.0-cp36-cp36m-linux_x86_64.whl size=394115 sha256=a4b5761320c67355f3ebd07292cd502278aeed49d5f38ed761fa7ef470498522
  Stored in directory: /root/.cache/pip/wheels/e7/f7/0f/b57055080cf8894906b3bd3616d2fc2bfd0b12d5161bcb24ac
Successfully built shap


In [0]:
import pandas as pd
import numpy as np

from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import gc

import shap

In [0]:

DEBUG = False

if DEBUG:
    NFOLDS = 2
    N_ESTIMATORS = 50
else:
    NFOLDS = 5
    N_ESTIMATORS = 2500

In [0]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [0]:
train = train.loc[:, train.max() != train.min()]

In [0]:
train['Y18'].isnull().sum(), train['Y18'].notnull().sum()
train_first_30 = train[train['Y18'].isnull()]
train_first_30.shape
train_later_3 = train[train['Y18'].notnull()]
train_later_3.shape

(432, 57)

In [0]:

X_train = train_first_30.loc[:, 'X00':'X39']
X_train

,X00,X01,X02,X03,X04,X05,X06,X07,X08,X09,X10,X11,X12,X13,X15,X17,X18,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39
0,9.7,988.8,1.2,0.6,0.0,1009.3,989.6,12.2,1009.9,1009.8,0.0,0.00,82.4,71.3,101.1,56.1,0.3,91.0,0.0,1002.5,1009.8,1.1,2.3,0.5,1008.7,12.3,1001.6,69.1,8.2,10.7,1010.1,0.00,256.4,0.0,77.2,62.6,0.0
1,9.3,988.9,1.7,1.9,0.0,1009.3,989.6,12.1,1010.0,1009.9,0.0,0.00,81.2,121.8,165.3,0.0,0.4,91.9,0.0,1002.5,1009.9,0.0,0.0,0.0,1008.7,12.1,1001.7,70.3,8.3,10.3,1010.1,0.00,215.4,0.0,77.3,63.5,0.0
2,9.4,989.0,1.1,2.3,0.0,1009.2,989.7,12.1,1010.1,1010.1,0.0,0.00,86.1,232.0,144.0,0.0,0.6,93.3,0.0,1002.4,1009.8,0.0,0.0,0.0,1008.6,12.1,1001.6,71.5,8.0,9.7,1010.0,0.00,235.2,0.0,77.3,63.9,0.0
3,9.4,988.9,1.5,0.7,0.0,1009.2,989.6,12.0,1010.0,1010.0,0.0,0.00,87.7,0.0,115.9,294.1,0.1,93.4,0.0,1002.4,1009.7,0.0,0.0,0.2,1008.7,12.1,1001.5,73.2,7.7,9.4,1010.1,0.00,214.0,0.0,77.5,64.5,0.0
4,9.2,988.9,0.8,1.7,0.0,1009.2,989.7,12.0,1010.1,1010.0,0.0,0.00,88.9,0.0,124.7,0.0,0.0,94.1,0.0,1002.4,1009.7,0.0,0.0,0.0,1008.7,12.0,1001.5,74.3,7.4,9.4,1010.1,0.00,174.9,0.0,78.0,65.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4315,19.5,987.8,1.7,0.4,0.0,1007.8,988.0,21.3,1007.8,1007.5,0.0,13.21,92.0,227.5,304.0,10.7,0.4,84.1,0.0,1001.2,1008.2,0.4,328.4,0.6,1006.6,21.6,1000.1,82.3,18.8,19.4,1008.2,16.68,272.9,0.0,82.3,74.3,0.0
4316,19.3,987.7,2.1,0.9,0.0,1007.8,988.1,21.3,1008.0,1007.4,0.0,13.21,92.0,259.4,309.1,0.7,0.3,85.7,0.0,1001.2,1008.2,0.2,346.7,1.1,1006.8,21.6,1000.1,85.8,18.6,18.5,1008.4,16.68,315.9,0.0,82.4,74.8,0.0
4317,19.5,987.8,0.9,1.3,0.0,1007.8,988.3,21.2,1008.1,1007.5,0.0,13.21,91.9,256.1,318.6,54.7,0.9,85.9,0.0,1001.2,1008.3,0.9,342.1,0.5,1006.8,21.5,1000.2,84.1,18.4,19.1,1008.4,16.68,343.3,0.0,82.8,75.4,0.0
4318,20.0,987.8,1.4,0.5,0.0,1007.8,988.3,21.1,1008.1,1007.5,0.0,13.21,92.1,203.9,327.7,116.1,0.4,83.7,0.0,1001.2,1008.1,0.7,345.5,0.5,1006.8,21.3,1000.0,85.4,18.2,19.0,1008.4,16.68,341.7,0.0,82.8,75.8,0.0


In [0]:
X_train = pd.concat([X_train, X_train], axis=0)
X_train = pd.concat([X_train, train_later_3.loc[:, 'X00':'X39']], axis=0)
X_train

,X00,X01,X02,X03,X04,X05,X06,X07,X08,X09,X10,X11,X12,X13,X15,X17,X18,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39
0,9.7,988.8,1.2,0.6,0.0,1009.3,989.6,12.2,1009.9,1009.8,0.0,0.00,82.4,71.3,101.1,56.1,0.3,91.0,0.0,1002.5,1009.8,1.1,2.3,0.5,1008.7,12.3,1001.6,69.1,8.2,10.7,1010.1,0.00,256.4,0.0,77.2,62.6,0.0
1,9.3,988.9,1.7,1.9,0.0,1009.3,989.6,12.1,1010.0,1009.9,0.0,0.00,81.2,121.8,165.3,0.0,0.4,91.9,0.0,1002.5,1009.9,0.0,0.0,0.0,1008.7,12.1,1001.7,70.3,8.3,10.3,1010.1,0.00,215.4,0.0,77.3,63.5,0.0
2,9.4,989.0,1.1,2.3,0.0,1009.2,989.7,12.1,1010.1,1010.1,0.0,0.00,86.1,232.0,144.0,0.0,0.6,93.3,0.0,1002.4,1009.8,0.0,0.0,0.0,1008.6,12.1,1001.6,71.5,8.0,9.7,1010.0,0.00,235.2,0.0,77.3,63.9,0.0
3,9.4,988.9,1.5,0.7,0.0,1009.2,989.6,12.0,1010.0,1010.0,0.0,0.00,87.7,0.0,115.9,294.1,0.1,93.4,0.0,1002.4,1009.7,0.0,0.0,0.2,1008.7,12.1,1001.5,73.2,7.7,9.4,1010.1,0.00,214.0,0.0,77.5,64.5,0.0
4,9.2,988.9,0.8,1.7,0.0,1009.2,989.7,12.0,1010.1,1010.0,0.0,0.00,88.9,0.0,124.7,0.0,0.0,94.1,0.0,1002.4,1009.7,0.0,0.0,0.0,1008.7,12.0,1001.5,74.3,7.4,9.4,1010.1,0.00,174.9,0.0,78.0,65.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4747,19.9,987.6,0.9,0.8,0.0,1006.9,987.7,21.7,1007.5,1007.4,2.5,23.88,95.3,333.1,274.9,275.5,0.9,89.7,10.5,1000.4,1007.5,0.0,0.0,1.2,1006.0,23.8,999.4,89.9,17.7,19.1,1007.5,22.16,218.6,0.0,82.3,58.6,0.5
4748,19.9,987.6,0.5,0.7,0.0,1006.8,987.7,21.6,1007.5,1007.4,2.5,23.88,95.6,216.4,277.3,247.0,1.0,90.1,10.5,1000.3,1007.5,0.0,0.0,1.3,1006.0,23.6,999.4,91.3,17.7,19.2,1007.5,22.16,161.7,0.0,82.5,59.1,0.5
4749,19.7,987.7,0.9,0.6,0.0,1006.9,987.6,21.4,1007.4,1007.5,2.5,23.88,95.9,127.8,256.1,299.1,0.6,90.8,10.5,1000.4,1007.6,0.0,0.0,0.2,1006.0,23.6,999.5,90.2,17.8,19.2,1007.5,22.16,254.2,0.0,83.0,58.9,0.5
4750,19.4,987.7,0.9,0.8,0.0,1006.9,987.8,21.3,1007.6,1007.5,2.5,23.88,95.9,323.8,255.1,0.0,0.6,91.5,10.5,1000.4,1007.6,0.0,0.0,0.1,1006.0,23.3,999.5,90.1,17.7,19.3,1007.6,22.16,300.0,0.0,83.2,59.8,0.5


In [0]:
#표준화 함수 생성
def standardization(df):
    mean = np.mean(df)
    std = np.std(df)
    norm = (df - mean) / (std - 1e-07)
    return norm, mean, std

In [0]:
# 학습데이터 표준화
X_train_norm, mean, std = standardization(X_train)

In [0]:
y_train = train_first_30.loc[:, 'Y16']
y_train = pd.concat([y_train, y_train], axis=0)
y_train = pd.concat([y_train, train_later_3.loc[:, 'Y18']], axis=0)
y_train

0        8.0
1        7.5
2        7.5
3        7.5
4        7.5
        ... 
4747    21.5
4748    21.5
4749    21.5
4750    21.5
4751    21.0
Length: 9072, dtype: float64

In [0]:

X_test = test.loc[:, 'X00':'X39']

# 모든 값이 동일한 column 제거
X_test = X_test.loc[:, X_test.max() != X_test.min()]

# 테스트데이터 표준화
X_test_norm = (X_test - mean) / (std - 1e-07)

In [0]:
# ID로 시간 변수 생성
minute = pd.Series((X_train_norm.index%144).astype(int))
hour= pd.Series((X_train_norm.index%144/6).astype(int))

# 삼각함수를 이용한 시간변수 생성
min_in_day = 24*6
hour_in_day = 24

minute_sin = np.sin(np.pi*minute/min_in_day) 
minute_cos = np.cos(np.pi*minute/min_in_day)

hour_sin  = np.sin(np.pi*hour/hour_in_day)
hour_cos  = np.cos(np.pi*hour/hour_in_day)


In [0]:

X_train_norm['minute_sin'] = minute_sin
X_train_norm['minute_cos'] = minute_cos

X_train_norm['hour_sin'] = hour_sin
X_train_norm['hour_cos'] = hour_cos
X_train_norm

,X00,X01,X02,X03,X04,X05,X06,X07,X08,X09,X10,X11,X12,X13,X15,X17,X18,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,minute_sin,minute_cos,hour_sin,hour_cos
0,-2.200858,0.021446,-0.302269,-0.901206,-0.294097,0.250176,0.140271,-2.125617,0.278398,0.359492,-0.308606,-1.009584,0.641257,-0.914050,-0.734332,-0.899913,-0.932557,0.889403,-0.280801,0.182418,0.242728,0.107266,-1.038180,-0.692139,0.296425,-2.280770,0.198271,0.167066,-2.071975,-1.731011,0.262630,-1.012858,0.492439,-0.281954,0.584950,0.253448,-0.239790,0.000000,1.000000,0.000000,1.000000
1,-2.285482,0.050911,0.160625,0.473953,-0.294097,0.250176,0.140271,-2.148344,0.306617,0.387481,-0.308606,-1.009584,0.587833,-0.400739,-0.072450,-1.387470,-0.846263,0.930206,-0.280801,0.182418,0.271742,-0.897265,-1.055778,-1.091776,0.296425,-2.327830,0.227499,0.222209,-2.054006,-1.806372,0.262630,-1.012858,0.076828,-0.281954,0.590018,0.302031,-0.239790,0.021815,0.999762,0.000000,1.000000
2,-2.264326,0.080376,-0.394848,0.897079,-0.294097,0.220989,0.169400,-2.148344,0.334836,0.443458,-0.308606,-1.009584,0.805978,0.719397,-0.292047,-1.387470,-0.673675,0.993677,-0.280801,0.153012,0.242728,-0.897265,-1.055778,-1.091776,0.267662,-2.327830,0.198271,0.277353,-2.107912,-1.919412,0.233818,-1.012858,0.277538,-0.281954,0.590018,0.323624,-0.239790,0.043619,0.999048,0.000000,1.000000
3,-2.264326,0.050911,-0.024533,-0.795425,-0.294097,0.220989,0.140271,-2.171071,0.306617,0.415469,-0.308606,-1.009584,0.877209,-1.638784,-0.581749,1.168513,-1.105144,0.998210,-0.280801,0.153012,0.213715,-0.897265,-1.055778,-0.931921,0.296425,-2.327830,0.169043,0.355474,-2.161819,-1.975933,0.262630,-1.012858,0.062636,-0.281954,0.600155,0.356013,-0.239790,0.065403,0.997859,0.000000,1.000000
4,-2.306638,0.050911,-0.672584,0.262390,-0.294097,0.220989,0.169400,-2.171071,0.334836,0.415469,-0.308606,-1.009584,0.930632,-1.638784,-0.491024,-1.387470,-1.191438,1.029946,-0.280801,0.153012,0.213715,-0.897265,-1.055778,-1.091776,0.296425,-2.351360,0.169043,0.406023,-2.215725,-1.975933,0.262630,-1.012858,-0.333715,-0.281954,0.625499,0.383004,-0.239790,0.087156,0.996195,0.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4747,-0.042946,-0.332135,-0.580005,-0.689643,-0.294097,-0.450307,-0.413183,0.033452,-0.398857,-0.312228,0.427879,1.177892,1.215556,1.747035,1.057492,1.006863,-0.414794,0.830466,2.250951,-0.435113,-0.424580,-0.897265,-1.055778,-0.132648,-0.480176,0.425161,-0.444743,1.122894,-0.364936,-0.148445,-0.486473,0.927250,0.109266,-0.281954,0.843450,0.037521,-0.133855,0.108867,-0.994056,0.130526,-0.991445
4748,-0.042946,-0.332135,-0.950320,-0.795425,-0.294097,-0.479494,-0.413183,0.010725,-0.398857,-0.312228,0.427879,1.177892,1.228912,0.560829,1.082235,0.759173,-0.328500,0.848600,2.250951,-0.464520,-0.424580,-0.897265,-1.055778,-0.052721,-0.480176,0.378101,-0.444743,1.187229,-0.364936,-0.129604,-0.486473,0.927250,-0.467522,-0.281954,0.853587,0.064512,-0.133855,0.087156,-0.996195,0.130526,-0.991445
4749,-0.085258,-0.302670,-0.580005,-0.901206,-0.294097,-0.450307,-0.442312,-0.034729,-0.427076,-0.284240,0.427879,1.177892,1.242267,-0.339752,0.863670,1.211967,-0.673675,0.880336,2.250951,-0.435113,-0.395567,-0.897265,-1.055778,-0.931921,-0.480176,0.378101,-0.415516,1.136680,-0.346967,-0.129604,-0.486473,0.927250,0.470138,-0.281954,0.878930,0.053716,-0.133855,0.065403,-0.997859,0.130526,-0.991445
4750,-0.148726,-0.302670,-0.580005,-0.689643,-0.294097,-0.450307,-0.384054,-0.057456,-0.370638,-0.284240,0.427879,1.177892,1.242267,1.652504,0.853360,-1.387470,-0.673675,0.912071,2.250951,-0.435113,-0.395567,-0.897265,-1.055778,-1.011848,-0.480176,0.307511,-0.415516,1.132085,-0.364936,-0.110764,-0.457661,0.927250,0.934407,-0.281954,0.889067,0.102299,-0.133855,0.043619,-0.999048,0.130526,-0.991445


In [0]:

# ID로 시간 변수 생성
minute = pd.Series((test.id%144).astype(int))
hour= pd.Series((test.id%144/6).astype(int))

# 삼각함수를 이용한 시간변수 생성
min_in_day = 24*6
hour_in_day = 24

minute_sin = np.sin(np.pi*minute/min_in_day) 
minute_cos = np.cos(np.pi*minute/min_in_day)

hour_sin  = np.sin(np.pi*hour/hour_in_day)
hour_cos  = np.cos(np.pi*hour/hour_in_day)

In [0]:
X_test_norm['minute_sin'] = minute_sin
X_test_norm['minute_cos'] = minute_cos

X_test_norm['hour_sin'] = hour_sin
X_test_norm['hour_cos'] = hour_cos
X_test_norm

,X00,X01,X02,X03,X04,X05,X06,X07,X08,X09,X10,X11,X12,X13,X15,X17,X18,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,minute_sin,minute_cos,hour_sin,hour_cos
0,-0.296818,-0.391065,-0.024533,-0.160736,-0.294097,-0.479494,-0.413183,-0.102911,-0.398857,-0.368205,-0.308606,-1.009584,1.251171,1.752117,0.281172,-1.074599,-0.242207,1.057148,-0.280801,-0.464520,-0.395567,-0.897265,-1.055778,-1.011848,-0.480176,0.283982,-0.415516,1.191824,-0.400874,-0.129604,-0.457661,-1.012858,-0.133005,-0.281954,0.929616,0.107697,-0.239790,0.000000,1.000000,0.000000,1.000000
1,-0.275662,-0.391065,-0.394848,-0.266518,-0.294097,-0.508680,-0.413183,-0.080183,-0.398857,-0.368205,-0.308606,-1.009584,1.255623,1.771430,0.223438,-0.277647,-0.673675,1.084349,-0.280801,-0.493926,-0.424580,-0.897265,-1.055778,-0.851994,-0.508939,0.260452,-0.444743,1.201015,-0.418843,-0.148445,-0.486473,-1.012858,-0.351962,-0.281954,0.929616,0.140086,-0.239790,0.021815,0.999762,0.000000,1.000000
2,-0.233350,-0.391065,-0.209690,-0.266518,-0.294097,-0.537867,-0.442312,-0.080183,-0.427076,-0.368205,-0.308606,-1.009584,1.264527,-1.638784,0.005904,-0.874709,-1.191438,1.020878,-0.280801,-0.523332,-0.453593,-0.897265,-1.055778,-0.532285,-0.508939,0.260452,-0.473971,1.219396,-0.454780,-0.167285,-0.486473,-1.012858,0.007897,-0.281954,0.980303,0.156281,-0.239790,0.043619,0.999048,0.000000,1.000000
3,-0.296818,-0.391065,0.716097,-0.689643,-0.294097,-0.537867,-0.442312,-0.102911,-0.427076,-0.368205,-0.308606,-1.009584,1.268979,0.931836,-0.194104,-0.910342,-0.846263,1.043547,-0.280801,-0.523332,-0.453593,-0.897265,-1.055778,-0.612212,-0.537702,0.189862,-0.473971,1.265350,-0.454780,-0.204965,-0.515284,-1.012858,-0.231333,-0.281954,1.020852,0.177873,-0.239790,0.065403,0.997859,0.000000,1.000000
4,-0.360286,-0.391065,-0.394848,-0.795425,-0.294097,-0.508680,-0.413183,-0.148365,-0.398857,-0.368205,-0.308606,-1.009584,1.273431,0.782417,0.015182,-0.963356,-0.501088,1.102484,-0.280801,-0.493926,-0.482607,-0.897265,-1.055778,-0.532285,-0.508939,0.189862,-0.503199,1.233182,-0.418843,-0.186125,-0.486473,-1.012858,-0.456371,-0.281954,1.096881,0.204864,-0.239790,0.087156,0.996195,0.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11515,1.184102,1.317909,2.197357,2.483800,-0.294097,0.950658,1.363694,1.078896,1.237842,1.143166,-0.308606,0.556825,0.895017,-0.971988,-0.729177,0.148205,0.361850,0.268294,-0.280801,0.976387,1.142143,0.563871,0.243446,2.504952,1.101790,1.342824,1.162793,0.511715,0.695224,0.850080,1.098167,0.699620,-0.387441,-0.281954,0.711665,0.517958,-0.133855,0.108867,-0.994056,0.130526,-0.991445
11516,1.162946,1.317909,1.364149,1.637549,-0.294097,0.950658,1.363694,1.078896,1.237842,1.143166,-0.308606,0.556825,0.334073,-1.111243,-0.902380,0.244674,1.311081,0.304563,-0.280801,0.976387,1.142143,0.837834,0.007015,2.664806,1.073027,1.319294,1.162793,0.516311,1.000694,0.850080,1.069356,0.699620,-0.323578,-0.281954,0.706597,0.539550,-0.133855,0.087156,-0.996195,0.130526,-0.991445
11517,1.162946,1.317909,2.104778,1.108641,-0.294097,1.009031,1.334565,1.056169,1.209623,1.171155,-0.308606,0.556825,0.423112,-1.344011,-0.699279,0.218601,1.311081,0.340832,-0.280801,1.035199,1.171157,1.203118,0.066696,2.744734,1.073027,1.295765,1.192021,0.534692,0.928819,0.831240,1.069356,0.699620,-0.031637,-0.281954,0.686322,0.561143,-0.133855,0.065403,-0.997859,0.130526,-0.991445
11518,1.141790,1.317909,0.808676,1.425986,-0.294097,0.979845,1.334565,1.033442,1.209623,1.143166,-0.308606,0.556825,0.271746,-1.333847,-0.714744,0.244674,1.742550,0.377101,-0.280801,1.005793,1.171157,0.655192,0.065166,1.705679,1.073027,1.248705,1.192021,0.543883,1.000694,0.812400,1.069356,0.699620,-0.340811,-0.281954,0.731940,0.588134,-0.133855,0.043619,-0.999048,0.130526,-0.991445


In [0]:

X_train_norm_Y18 = X_train_norm.iloc[4320+4320:, :]

y_train_Y18 = y_train[4320+4320:]

In [0]:

# 실제값과 예측값의 차이가 1보다 작은 건 무시하는 Custom MSE Metric
def mse_ignoring_diff_less_than_one(y_pred, y_true):

    diff = abs(y_true - y_pred)
    mask_less_than_one = np.array([0 if x < 1 else 1 for x in diff])

    y_pred = mask_less_than_one * y_pred
    y_true = mask_less_than_one *y_true
    
    score =  mean_squared_error(y_true, y_pred)
    
    return 'score', score, False

In [0]:
# Y15, Y16, Y18을 활용한 훈련데이터 세트 모델링

feature_importance = pd.DataFrame()
feature_importance['feature'] = X_train_norm.columns

folds = KFold(n_splits=NFOLDS, shuffle=True)
y_preds = np.zeros(X_test_norm.shape[0])

model = XGBRegressor(n_estimators=N_ESTIMATORS,
                       max_depth=3,
                       learning_rate=0.007,
                     boost_from_average=False,
                       subsample=1,
                     loss_function='rmse',
                     scale_pos_weight=10,
                     coal_sample_by_tree=0.75,
                     min_child_weight=4,
                     feval=mse_ignoring_diff_less_than_one,
                       random_state=47)

for fold_n, (train_index, valid_index) in enumerate(folds.split(X_train_norm, y_train)):
    print('Fold: ', fold_n+1)
    X_train_fold, X_valid_fold = X_train_norm.iloc[train_index], X_train_norm.iloc[valid_index]
    y_train_fold, y_valid_fold = y_train.iloc[train_index], y_train.iloc[valid_index]
    
    evals = [(X_train_fold, y_train_fold), (X_valid_fold,  y_valid_fold)]
    model.fit(X_train_fold, 
              y_train_fold, 
              eval_set=evals,
              eval_metric='mae',
              verbose=100,
              early_stopping_rounds=50
             )
    
    shap_values = shap.TreeExplainer(model).shap_values(X_train_fold)
    fold_importance = pd.DataFrame()
    fold_importance['feature'] = X_train_norm.columns
    fold_importance['shap_values'] = abs(np.array(shap_values)[:, :].mean(0))
    fold_importance['feat_importance'] = model.feature_importances_
    feature_importance = pd.concat([feature_importance, fold_importance], sort=False)
        
    y_preds += model.predict(X_test_norm) / NFOLDS
    
    del X_train_fold, X_valid_fold, y_train_fold, y_valid_fold
    gc.collect()

feature_importance = feature_importance.groupby('feature').mean().sort_values('shap_values', ascending=False).reset_index()

Fold:  1
[12:54:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-mae:22.4318	validation_1-mae:22.7095
Multiple eval metrics have been passed: 'validation_1-mae' will be used for early stopping.

Will train until validation_1-mae hasn't improved in 50 rounds.


Series.base is deprecated and will be removed in a future version


[100]	validation_0-mae:11.121	validation_1-mae:11.2527
[200]	validation_0-mae:5.52449	validation_1-mae:5.5872
[300]	validation_0-mae:2.77996	validation_1-mae:2.81837
[400]	validation_0-mae:1.55589	validation_1-mae:1.59771
[500]	validation_0-mae:1.09725	validation_1-mae:1.14729
[600]	validation_0-mae:0.940957	validation_1-mae:0.995321
[700]	validation_0-mae:0.877701	validation_1-mae:0.932122
[800]	validation_0-mae:0.845329	validation_1-mae:0.900706
[900]	validation_0-mae:0.823335	validation_1-mae:0.878732
[1000]	validation_0-mae:0.80752	validation_1-mae:0.864561
[1100]	validation_0-mae:0.793807	validation_1-mae:0.852995
[1200]	validation_0-mae:0.780038	validation_1-mae:0.841309
[1300]	validation_0-mae:0.76736	validation_1-mae:0.830843
[1400]	validation_0-mae:0.756156	validation_1-mae:0.82143
[1500]	validation_0-mae:0.744725	validation_1-mae:0.811104
[1600]	validation_0-mae:0.733194	validation_1-mae:0.801171
[1700]	validation_0-mae:0.723274	validation_1-mae:0.791907
[1800]	validation_0-m

Setting feature_perturbation = "tree_path_dependent" because no background data was given.


Fold:  2
[12:55:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-mae:22.4651	validation_1-mae:22.581
Multiple eval metrics have been passed: 'validation_1-mae' will be used for early stopping.

Will train until validation_1-mae hasn't improved in 50 rounds.
[100]	validation_0-mae:11.1369	validation_1-mae:11.2256
[200]	validation_0-mae:5.53373	validation_1-mae:5.59974
[300]	validation_0-mae:2.7872	validation_1-mae:2.85022
[400]	validation_0-mae:1.55999	validation_1-mae:1.60311
[500]	validation_0-mae:1.1005	validation_1-mae:1.12813
[600]	validation_0-mae:0.942643	validation_1-mae:0.965434
[700]	validation_0-mae:0.878469	validation_1-mae:0.903311
[800]	validation_0-mae:0.846248	validation_1-mae:0.876486
[900]	validation_0-mae:0.826195	validation_1-mae:0.860623
[1000]	validation_0-mae:0.809148	validation_1-mae:0.846147
[1100]	validation_0-mae:0.794076	validation_1-mae:0.833131
[1200]	validation_0-mae:0.

In [0]:
# Y18만 활용한 훈련데이터 세트 모델링

feature_importance_Y18 = pd.DataFrame()
feature_importance_Y18['feature'] = X_train_norm_Y18.columns

folds = KFold(n_splits=NFOLDS, shuffle=True)
y_preds_Y18 = np.zeros(X_test_norm.shape[0])

model_Y18 = LGBMRegressor(n_estimators=N_ESTIMATORS,
                       boosting_type='gbdt',
                       num_leaves=70,
                       max_depth=-1,
                       learning_rate=0.01,
                       objective='regression',
                       sub_sample=0.8,
                       colsample_bytree=0.8,
                       min_data_in_leaf=50,
                       bagging_seed=11,
                       # metric=mse_ignoring_diff_less_than_one,
                       verbosity=-1,
                       random_state=47)

for fold_n, (train_index, valid_index) in enumerate(folds.split(X_train_norm_Y18, y_train_Y18)):
    print('Fold: ', fold_n+1)
    X_train_fold, X_valid_fold = X_train_norm_Y18.iloc[train_index], X_train_norm_Y18.iloc[valid_index]
    y_train_fold, y_valid_fold = y_train_Y18.iloc[train_index], y_train_Y18.iloc[valid_index]
    
    evals = [(X_train_fold, y_train_fold), (X_valid_fold,  y_valid_fold)]
    model_Y18.fit(X_train_fold, 
              y_train_fold, 
              eval_metric=mse_ignoring_diff_less_than_one,
              eval_set=evals,
              early_stopping_rounds=50,
              verbose=100
             )
    
    shap_values = shap.TreeExplainer(model_Y18.booster_).shap_values(X_train_fold)
    fold_importance = pd.DataFrame()
    fold_importance['feature'] = X_train_norm_Y18.columns
    fold_importance['shap_values'] = abs(np.array(shap_values)[:, :].mean(0))
    fold_importance['feat_importance'] = model_Y18.feature_importances_
    feature_importance_Y18 = pd.concat([feature_importance_Y18, fold_importance], sort=False)
        
    y_preds_Y18 += model_Y18.predict(X_test_norm) / NFOLDS
    
    del X_train_fold, X_valid_fold, y_train_fold, y_valid_fold
    gc.collect()

feature_importance_Y18 = feature_importance_Y18.groupby('feature').mean().sort_values('shap_values', ascending=False).reset_index()

Fold:  1
Training until validation scores don't improve for 50 rounds.
[100]	training's l2: 7.35029	training's score: 7.28258	valid_1's l2: 7.80987	valid_1's score: 7.73032
[200]	training's l2: 1.87515	training's score: 1.62317	valid_1's l2: 2.44658	valid_1's score: 2.24457
[300]	training's l2: 0.86787	training's score: 0.694433	valid_1's l2: 1.49211	valid_1's score: 1.34453
[400]	training's l2: 0.651333	training's score: 0.505478	valid_1's l2: 1.34263	valid_1's score: 1.18811
[500]	training's l2: 0.535125	training's score: 0.391608	valid_1's l2: 1.25113	valid_1's score: 1.10721
[600]	training's l2: 0.435584	training's score: 0.300137	valid_1's l2: 1.15842	valid_1's score: 1.02982
[700]	training's l2: 0.373642	training's score: 0.242585	valid_1's l2: 1.08726	valid_1's score: 0.954319
[800]	training's l2: 0.330366	training's score: 0.196728	valid_1's l2: 1.0412	valid_1's score: 0.903073
[900]	training's l2: 0.296919	training's score: 0.170402	valid_1's l2: 1.00794	valid_1's score: 0.865

Setting feature_perturbation = "tree_path_dependent" because no background data was given.


Fold:  2
Training until validation scores don't improve for 50 rounds.
[100]	training's l2: 7.47074	training's score: 7.39557	valid_1's l2: 6.38904	valid_1's score: 6.33639
[200]	training's l2: 1.86643	training's score: 1.62743	valid_1's l2: 1.68525	valid_1's score: 1.41353
[300]	training's l2: 0.852425	training's score: 0.647484	valid_1's l2: 0.989837	valid_1's score: 0.782084
Early stopping, best iteration is:
[320]	training's l2: 0.783196	training's score: 0.570837	valid_1's l2: 0.95237	valid_1's score: 0.716097


Setting feature_perturbation = "tree_path_dependent" because no background data was given.


Fold:  3
Training until validation scores don't improve for 50 rounds.
[100]	training's l2: 7.53932	training's score: 7.47546	valid_1's l2: 7.0198	valid_1's score: 6.96978
[200]	training's l2: 2.00793	training's score: 1.76973	valid_1's l2: 2.24618	valid_1's score: 2.03935
[300]	training's l2: 0.962877	training's score: 0.787001	valid_1's l2: 1.23976	valid_1's score: 1.08194
[400]	training's l2: 0.73106	training's score: 0.552249	valid_1's l2: 1.01653	valid_1's score: 0.863704
[500]	training's l2: 0.590093	training's score: 0.432547	valid_1's l2: 0.908597	valid_1's score: 0.754552
[600]	training's l2: 0.491557	training's score: 0.336119	valid_1's l2: 0.831094	valid_1's score: 0.700158
[700]	training's l2: 0.41553	training's score: 0.264853	valid_1's l2: 0.754594	valid_1's score: 0.59532
[800]	training's l2: 0.358515	training's score: 0.223699	valid_1's l2: 0.691218	valid_1's score: 0.531286
[900]	training's l2: 0.317303	training's score: 0.179628	valid_1's l2: 0.664282	valid_1's score:

Setting feature_perturbation = "tree_path_dependent" because no background data was given.


Fold:  4
Training until validation scores don't improve for 50 rounds.
[100]	training's l2: 7.3495	training's score: 7.30383	valid_1's l2: 7.0683	valid_1's score: 7.01198
[200]	training's l2: 1.95327	training's score: 1.72446	valid_1's l2: 1.97705	valid_1's score: 1.79013
[300]	training's l2: 0.959673	training's score: 0.7829	valid_1's l2: 1.05176	valid_1's score: 0.879726
[400]	training's l2: 0.737032	training's score: 0.600244	valid_1's l2: 0.855659	valid_1's score: 0.666087
[500]	training's l2: 0.626927	training's score: 0.493094	valid_1's l2: 0.773271	valid_1's score: 0.592048
[600]	training's l2: 0.504168	training's score: 0.373147	valid_1's l2: 0.686398	valid_1's score: 0.5331
[700]	training's l2: 0.436129	training's score: 0.309232	valid_1's l2: 0.641335	valid_1's score: 0.477338
[800]	training's l2: 0.387407	training's score: 0.271197	valid_1's l2: 0.61049	valid_1's score: 0.43836
[900]	training's l2: 0.346894	training's score: 0.227434	valid_1's l2: 0.588411	valid_1's score: 0

Setting feature_perturbation = "tree_path_dependent" because no background data was given.


Fold:  5
Training until validation scores don't improve for 50 rounds.
[100]	training's l2: 7.166	training's score: 7.09271	valid_1's l2: 10.0293	valid_1's score: 9.97071
[200]	training's l2: 2.00261	training's score: 1.77089	valid_1's l2: 3.11288	valid_1's score: 2.93532
[300]	training's l2: 1.0326	training's score: 0.856332	valid_1's l2: 1.64626	valid_1's score: 1.4727
[400]	training's l2: 0.796864	training's score: 0.628354	valid_1's l2: 1.28401	valid_1's score: 1.13824
[500]	training's l2: 0.645641	training's score: 0.484515	valid_1's l2: 1.06784	valid_1's score: 0.919988
[600]	training's l2: 0.527041	training's score: 0.369132	valid_1's l2: 0.928057	valid_1's score: 0.763367
[700]	training's l2: 0.446574	training's score: 0.298294	valid_1's l2: 0.844587	valid_1's score: 0.692334
[800]	training's l2: 0.383699	training's score: 0.244353	valid_1's l2: 0.773338	valid_1's score: 0.632104
[900]	training's l2: 0.338876	training's score: 0.201877	valid_1's l2: 0.719709	valid_1's score: 0.

Setting feature_perturbation = "tree_path_dependent" because no background data was given.


In [0]:
# Y18만 활용한 훈련데이터 세트 모델링

feature_importance_Y18 = pd.DataFrame()
feature_importance_Y18['feature'] = X_train_norm_Y18.columns

folds = KFold(n_splits=NFOLDS, shuffle=True)
y_preds_Y18 = np.zeros(X_test_norm.shape[0])

model_Y18 = XGBRegressor(n_estimators=N_ESTIMATORS,
                       max_depth=3,
                       learning_rate=0.007,
                     boost_from_average=False,
                       subsample=1,
                     loss_function='rmse',
                     scale_pos_weight=10,
                     coal_sample_by_tree=0.75,
                     min_child_weight=4,
                     feval=mse_ignoring_diff_less_than_one,
                       random_state=47)

for fold_n, (train_index, valid_index) in enumerate(folds.split(X_train_norm_Y18, y_train_Y18)):
    print('Fold: ', fold_n+1)
    X_train_fold, X_valid_fold = X_train_norm_Y18.iloc[train_index], X_train_norm_Y18.iloc[valid_index]
    y_train_fold, y_valid_fold = y_train_Y18.iloc[train_index], y_train_Y18.iloc[valid_index]
    
    evals = [(X_train_fold, y_train_fold), (X_valid_fold,  y_valid_fold)]
    model_Y18.fit(X_train_fold, 
              y_train_fold, 
              eval_set=evals,
              eval_metric='mae',
              verbose=100,
              early_stopping_rounds=50
             )
    
    shap_values = shap.TreeExplainer(model).shap_values(X_train_fold)
    fold_importance = pd.DataFrame()
    fold_importance['feature'] = X_train_norm_Y18.columns
    fold_importance['shap_values'] = abs(np.array(shap_values)[:, :].mean(0))
    fold_importance['feat_importance'] = model_Y18.feature_importances_
    feature_importance_Y18 = pd.concat([feature_importance_Y18, fold_importance], sort=False)
        
    y_preds_Y18 += model_Y18.predict(X_test_norm) / NFOLDS
    
    del X_train_fold, X_valid_fold, y_train_fold, y_valid_fold
    gc.collect()
xgb_importance_Y18 = feature_importance_Y18.groupby('feature').mean().sort_values('shap_values', ascending=False).reset_index()

Fold:  1
[13:09:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-mae:26.3427	validation_1-mae:24.6739
Multiple eval metrics have been passed: 'validation_1-mae' will be used for early stopping.

Will train until validation_1-mae hasn't improved in 50 rounds.


Series.base is deprecated and will be removed in a future version


[100]	validation_0-mae:13.1619	validation_1-mae:12.2843
[200]	validation_0-mae:6.59336	validation_1-mae:6.15126
[300]	validation_0-mae:3.3103	validation_1-mae:3.07175
[400]	validation_0-mae:1.66879	validation_1-mae:1.55032
[500]	validation_0-mae:0.856225	validation_1-mae:0.844916
[600]	validation_0-mae:0.475035	validation_1-mae:0.563813
[700]	validation_0-mae:0.328094	validation_1-mae:0.468963
[800]	validation_0-mae:0.275183	validation_1-mae:0.431947
[900]	validation_0-mae:0.252424	validation_1-mae:0.425447
[1000]	validation_0-mae:0.238272	validation_1-mae:0.422244
[1100]	validation_0-mae:0.227268	validation_1-mae:0.420808
[1200]	validation_0-mae:0.217471	validation_1-mae:0.4191
[1300]	validation_0-mae:0.207875	validation_1-mae:0.418064
[1400]	validation_0-mae:0.198604	validation_1-mae:0.415583
[1500]	validation_0-mae:0.189533	validation_1-mae:0.414453
[1600]	validation_0-mae:0.18137	validation_1-mae:0.41437
Stopping. Best iteration:
[1573]	validation_0-mae:0.183562	validation_1-mae:0.

Setting feature_perturbation = "tree_path_dependent" because no background data was given.


Fold:  2
[13:09:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-mae:26.1814	validation_1-mae:25.3106
Multiple eval metrics have been passed: 'validation_1-mae' will be used for early stopping.

Will train until validation_1-mae hasn't improved in 50 rounds.
[100]	validation_0-mae:13.0801	validation_1-mae:12.5863
[200]	validation_0-mae:6.55109	validation_1-mae:6.32093
[300]	validation_0-mae:3.2888	validation_1-mae:3.17111
[400]	validation_0-mae:1.65745	validation_1-mae:1.60433
[500]	validation_0-mae:0.850047	validation_1-mae:0.861916
[600]	validation_0-mae:0.487231	validation_1-mae:0.53837
[700]	validation_0-mae:0.343923	validation_1-mae:0.414502
[800]	validation_0-mae:0.282586	validation_1-mae:0.373419
[900]	validation_0-mae:0.254576	validation_1-mae:0.354249
[1000]	validation_0-mae:0.23674	validation_1-mae:0.344353
[1100]	validation_0-mae:0.224282	validation_1-mae:0.342616
[1200]	validation_0-mae:

In [0]:
ensembled_preds = 0.7 * y_preds + 0.3 * y_preds_Y18

submit = pd.DataFrame({'id':range(144*33, 144*113),
              'Y18':ensembled_preds})
submit.to_csv('xgb_result.csv', index = False)

In [0]:
y_preds

array([19.38420987, 19.45270252, 19.22534657, ..., 25.23366117,
       24.74242973, 24.62841558])

In [0]:
feature_importance.sort_values(by=['feat_importance'], axis=0,ascending=False).head(12)

,feature,shap_values,feat_importance
4,X00,0.074556,0.268419
9,X32,0.039561,0.217917
2,X31,0.082703,0.157755
3,X12,0.077013,0.124028
1,hour_sin,0.097856,0.090599
0,minute_sin,0.134485,0.059391
7,X07,0.047870,0.028075
5,X37,0.054796,0.011453
12,X30,0.017612,0.007596
39,X39,0.000408,0.003727


In [0]:
feature_importance_Y18.sort_values(by=['feat_importance'], axis=0,ascending=False).head(12)

,feature,shap_values,feat_importance
0,minute_sin,0.094806,502.6
3,X20,0.045751,370.4
5,X30,0.032781,321.8
11,X38,0.018446,306.8
7,X37,0.027134,291.2
2,hour_sin,0.071339,275.8
10,X28,0.019046,265.2
8,minute_cos,0.021015,237.4
15,X12,0.010304,218.8
4,X32,0.043028,201.2


In [0]:
xgb_importance_Y18.sort_values(by=['feat_importance'], axis=0,ascending=False).head(12)

,feature,shap_values,feat_importance
6,hour_sin,0.101902,0.316189
9,X12,0.061756,0.239774
11,X37,0.040174,0.194504
10,X30,0.059452,0.070944
19,X34,0.007591,0.025943
2,X31,0.661825,0.023315
1,X32,0.871539,0.022115
22,X08,0.004536,0.018023
4,minute_sin,0.251289,0.015405
28,X28,0.002707,0.014104
